In [1]:
# ============================
# 0. 구글 드라이브 마운트
# ============================
from google.colab import drive
drive.mount('/content/drive')

# (선택) 드라이브 구조 확인
!ls -lah /content/drive
!ls -lah "/content/drive/MyDrive"




# ============================
# 1. 패키지 설치
# ============================
!pip -q install moviepy requests
!pip -q install transformers accelerate librosa soundfile torchaudio



# ============================
# 2. import
# ============================
import os
from pathlib import Path
import mimetypes
import json
import time
import getpass

import numpy as np
import torch
import librosa
import soundfile as sf
import requests

from transformers import (
    pipeline,
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)

from collections import Counter
import csv
import math



# ============================
# 2-0. 한국어 알파벳 이름 → 이니셜 후처리 함수
# ============================
"""
예)
- '더블유 티 오 회의에서'  -> 'WTO 회의에서'
- '에스 케이 텔레콤'        -> 'SK 텔레콤'
- '에이 아이 모델'          -> 'AI 모델'
"""

LETTER_NAME_TO_CHAR = {
    "에이": "A",
    "비": "B",
    "씨": "C",
    "시": "C",
    "디": "D",
    "이": "E",
    "에프": "F",
    "지": "G",
    "쥐": "G",
    "에이치": "H",
    "아이": "I",
    "제이": "J",
    "케이": "K",
    "엘": "L",
    "엠": "M",
    "엔": "N",
    "오": "O",
    "피": "P",
    "큐": "Q",
    "아르": "R",
    "알": "R",
    "에스": "S",
    "티": "T",
    "유": "U",
    "브이": "V",
    "더블유": "W",
    "엑스": "X",
    "와이": "Y",
    "제트": "Z",
    "지드": "Z",
}


def normalize_acronyms_ko(text: str, min_letters: int = 2) -> str:
    """
    한국어로 발음된 알파벳 이름 시퀀스를 이니셜로 치환하는 함수.

    - min_letters: 몇 글자 이상 연속되었을 때 이니셜로 볼 것인지 (기본 2)

    예:
        "더블유 티 오 회의" -> "WTO 회의"
        "에이 아이 모델"   -> "AI 모델"
    """
    if not text.strip():
        return text

    tokens = text.split()
    result_tokens = []
    i = 0
    while i < len(tokens):
        j = i
        letters = []
        while j < len(tokens):
            raw = tokens[j]
            core = raw.rstrip(".,?!")
            suffix = raw[len(core):]
            char = LETTER_NAME_TO_CHAR.get(core)
            if not char:
                break
            letters.append((char, suffix))
            j += 1

        if len(letters) >= min_letters:
            acronym = "".join([c for c, _ in letters]) + letters[-1][1]
            result_tokens.append(acronym)
            i = j
        else:
            result_tokens.append(tokens[i])
            i += 1

    return " ".join(result_tokens)



# ============================
# 2-1. 리턴제로 STT 클라이언트 정의
# ============================
class RTZROpenAPIClient:
    """
    리턴제로 파일 STT용 최소 클라이언트
    - /v1/authenticate : JWT 발급
    - /v1/transcribe   : 파일 STT 요청
    - /v1/transcribe/{id} : 결과 폴링
    """
    def __init__(self, client_id: str, client_secret: str,
                 base_url: str = "https://openapi.vito.ai"):
        self.base_url = base_url.rstrip("/")
        self.client_id = client_id
        self.client_secret = client_secret

        if not self.client_id or not self.client_secret:
            raise ValueError("RTZR CLIENT_ID / CLIENT_SECRET가 비었습니다.")

        self._sess = requests.Session()
        self._token = None

    @property
    def token(self) -> str:
        """현재 JWT 토큰을 반환, 없거나 오래되면 /v1/authenticate로 갱신"""
        if self._token is None or self._token.get("expire_at", 0) < time.time() - 1800:
            resp = self._sess.post(
                f"{self.base_url}/v1/authenticate",
                data={
                    "client_id": self.client_id,
                    "client_secret": self.client_secret,
                },
            )
            resp.raise_for_status()
            self._token = resp.json()

        access = self._token.get("access_token")
        if not access:
            raise RuntimeError("authenticate 응답에 access_token이 없습니다.")
        return access

    def _auth_headers(self):
        return {"Authorization": f"Bearer {self.token}"}

    def transcribe_file(self, file_path: str, config: dict) -> dict:
        """ /v1/transcribe 로 파일 STT 요청 """
        url = f"{self.base_url}/v1/transcribe"
        with open(file_path, "rb") as f:
            files = {"file": (os.path.basename(file_path), f)}
            data = {"config": json.dumps(config)}
            resp = self._sess.post(url, headers=self._auth_headers(),
                                   files=files, data=data)
            resp.raise_for_status()
            return resp.json()  # {"id": "..."} 형태

    def get_transcription(self, transcribe_id: str) -> dict:
        url = f"{self.base_url}/v1/transcribe/{transcribe_id}"
        resp = self._sess.get(url, headers=self._auth_headers())
        resp.raise_for_status()
        return resp.json()

    def wait_for_result(self, transcribe_id: str,
                        poll_interval_sec: int = 5,
                        timeout_sec: int = 3600) -> dict:
        """폴링하면서 completed/failed 될 때까지 기다렸다가 결과 리턴"""
        deadline = time.time() + timeout_sec
        while True:
            if time.time() > deadline:
                raise TimeoutError("STT 결과 대기 중 timeout")

            result = self.get_transcription(transcribe_id)
            status = result.get("status")
            if status in ("completed", "failed"):
                return result

            time.sleep(poll_interval_sec)



# ============================
# 3. 입력 미디어 설정 (비디오 or 오디오)
# ============================
media_path = "/content/drive/MyDrive/data_MP3/PMCT_42min.mp3"        # .mp4 / .mov / .mp3 / .wav 등 OK
desired_audio_path = "/content/drive/MyDrive/data_MP3/extracted_audio.mp3"  # 비디오일 때만 사용

def is_audio(path: str) -> bool:
    mt, _ = mimetypes.guess_type(path)
    return (mt or "").startswith("audio")

if is_audio(media_path):
    audio_path = media_path
    print(f"[SKIP extract] Detected audio file. Use as-is: {audio_path}")
else:
    from moviepy.editor import VideoFileClip
    Path(Path(desired_audio_path).parent).mkdir(parents=True, exist_ok=True)
    clip = VideoFileClip(media_path)
    if clip.audio is None:
        raise RuntimeError("이 비디오에는 오디오 트랙이 없습니다.")
    clip.audio.write_audiofile(desired_audio_path)
    clip.close()
    audio_path = desired_audio_path
    print(f"[EXTRACT] Audio saved to: {audio_path}")



# ============================
# 4. 리턴제로 기반 텍스트 인식
# ============================
clean_audio_path = "/content/drive/MyDrive/data_MP3/cleaned_sample.wav"

# 4-1) 오디오 전처리
y, sr = librosa.load(audio_path, sr=16000, mono=True)
raw_duration = len(y) / sr
print(f"[DEBUG] 원본 오디오 duration = {raw_duration:.2f} s")

y, _ = librosa.effects.trim(y, top_db=30)
peak = np.max(np.abs(y))
if peak > 0:
    y = y / peak

Path(clean_audio_path).parent.mkdir(parents=True, exist_ok=True)
sf.write(clean_audio_path, y, 16000)
audio_for_asr = clean_audio_path

clean_duration = librosa.get_duration(path=audio_for_asr)
print(f"[DEBUG] 정제된 오디오 duration = {clean_duration:.2f} s")

# 4-2) RTZR API 키
if "RTZR_CLIENT_ID" in os.environ and "RTZR_CLIENT_SECRET" in os.environ:
    RTZR_CLIENT_ID = os.environ["RTZR_CLIENT_ID"]
    RTZR_CLIENT_SECRET = os.environ["RTZR_CLIENT_SECRET"]
    print("[INFO] 환경변수에서 RTZR CLIENT_ID/SECRET을 읽었습니다.")
else:
    RTZR_CLIENT_ID = input("RTZR CLIENT_ID를 입력하세요: ").strip()
    RTZR_CLIENT_SECRET = getpass.getpass(
        "RTZR CLIENT_SECRET를 입력하세요 (입력 시 화면에 표시되지 않습니다): "
    ).strip()

client = RTZROpenAPIClient(RTZR_CLIENT_ID, RTZR_CLIENT_SECRET)

# 4-3) STT 설정
SPEAKER_COUNT = 0  # 0이면 화자 수 자동 추정

config = {
    "model_name": "sommers",
    "domain": "GENERAL",
    "use_diarization": True,
    "diarization": {"spk_count": SPEAKER_COUNT},
    "use_paragraph_splitter": True,
    "paragraph_splitter": {"max": 80},
    "use_itn": False,
    "use_disfluency_filter": False,
    "use_profanity_filter": False,
}

print("[DEBUG] RTZR STT config:")
print(json.dumps(config, ensure_ascii=False, indent=2))

# 4-4) STT 요청 + 결과
submit_resp = client.transcribe_file(audio_for_asr, config)
transcribe_id = submit_resp.get("id")
print(f"[INFO] STT 작업 ID: {transcribe_id}")

if not transcribe_id:
    print("[ERROR] transcribe_id가 없습니다. 응답 전체:")
    print(json.dumps(submit_resp, ensure_ascii=False, indent=2))
    raise SystemExit

print("[INFO] 결과 대기 중 (5초 간격 폴링)...")
rtzr_result = client.wait_for_result(transcribe_id, poll_interval_sec=5, timeout_sec=3600)

print(f"[DEBUG] RTZR status = {rtzr_result.get('status')}")

# 4-5) 결과 → segments_to_use
utterances = rtzr_result.get("results", {}).get("utterances", [])
print(f"[DEBUG] 총 utterance 개수 = {len(utterances)}")

segments_to_use = []
for utt in utterances:
    start_ms = int(utt.get("start_at", 0))
    dur_ms = int(utt.get("duration", 0))
    start_s = start_ms / 1000.0
    end_s = (start_ms + dur_ms) / 1000.0

    text = utt.get("msg", "").strip().replace("\n", " ")
    text = normalize_acronyms_ko(text)   # 더블유 티 오 → WTO 등
    spk = utt.get("spk", 0)

    segments_to_use.append({
        "start": start_s,
        "end": end_s,
        "text": text,
        "speaker": spk,
    })

if segments_to_use:
    last_end = segments_to_use[-1]["end"]
    print(f"[DEBUG] 마지막 세그먼트 end = {last_end:.2f} s")

    print("\n[DEBUG] 처음 5개 세그먼트:")
    for seg in segments_to_use[:5]:
        print(
            f"  [{seg['start']:.2f}-{seg['end']:.2f}] "
            f"Speaker={seg['speaker']} :: {seg['text']}"
        )

    print("\n[DEBUG] 마지막 5개 세그먼트:")
    for seg in segments_to_use[-5:]:
        print(
            f"  [{seg['start']:.2f}-{seg['end']:.2f}] "
            f"Speaker={seg['speaker']} :: {seg['text']}"
        )

for seg in segments_to_use:
    print(f"[{seg['start']:.2f}–{seg['end']:.2f}] (spk={seg['speaker']}) {seg['text']}")

full_text = " ".join(seg["text"] for seg in segments_to_use)
print("Full text:\n", full_text)

output_txt_path = '/content/drive/MyDrive/output/sample.txt'
Path(output_txt_path).parent.mkdir(parents=True, exist_ok=True)
with open(output_txt_path, 'w', encoding="utf-8") as f:
    for seg in segments_to_use:
        f.write(f"[{seg['start']:.2f}–{seg['end']:.2f}] (spk={seg['speaker']}) {seg['text']}\n")

print(f"\nTranscription saved to {output_txt_path}")



# ============================
# 5. 감정(SER) 분석 + 파형 특징
# ============================
y_ser, sr_ser = librosa.load(audio_for_asr, sr=16000, mono=True)
duration_ser = len(y_ser) / 16000.0
print(f"Loaded for SER: {audio_for_asr}, sr=16000, duration={duration_ser:.2f}s")

model_id = "superb/hubert-large-superb-er"
config_ser = AutoConfig.from_pretrained(model_id)
print("SER labels:", config_ser.id2label)

ser = pipeline(task="audio-classification", model=model_id, top_k=None)

def slice_audio(y, sr, start_s, end_s):
    i0 = int(max(0, start_s) * sr)
    i1 = int(min(len(y) / sr, end_s) * sr)
    return y[i0:i1]

def predict_emotion(wave_1d, sr=16000):
    if len(wave_1d) < int(sr * 0.3):
        return {"label": "unknown", "score": 0.0, "probs": {}}
    out = ser({"array": wave_1d, "sampling_rate": sr})
    top = max(out, key=lambda d: d["score"])
    probs = {d["label"]: float(d["score"]) for d in out}
    return {"label": top["label"], "score": float(top["score"]), "probs": probs}

def extract_acoustic_features(wave_1d, sr, text):
    """
    회의용 감정 디테일을 위해 파형에서 기본적인 특징 추출:
    - rms_energy: 에너지(크기)
    - zcr: zero-crossing-rate (거칠기/잡음성)
    - duration: 길이
    - speaking_rate: 말 속도(초당 글자 수)
    - f0_mean, f0_std, voiced_ratio: 피치 통계
    """
    if len(wave_1d) == 0 or sr <= 0:
        return {
            "rms_energy": 0.0,
            "zcr": 0.0,
            "duration": 0.0,
            "speaking_rate": 0.0,
            "f0_mean": 0.0,
            "f0_std": 0.0,
            "voiced_ratio": 0.0,
        }

    duration = len(wave_1d) / sr
    rms = float(np.mean(librosa.feature.rms(y=wave_1d)))
    zcr = float(np.mean(librosa.feature.zero_crossing_rate(y=wave_1d)))

    char_len = len(text.replace(" ", ""))
    speaking_rate = char_len / max(duration, 1e-3)

    f0_mean, f0_std, voiced_ratio = 0.0, 0.0, 0.0
    try:
        if duration >= 0.25:
            f0, vflag, _ = librosa.pyin(
                wave_1d, fmin=50, fmax=500, sr=sr
            )
            if f0 is not None:
                vflag = vflag.astype(bool)
                valid_f0 = f0[vflag]
                if valid_f0.size > 0:
                    f0_mean = float(np.nanmean(valid_f0))
                    f0_std = float(np.nanstd(valid_f0))
                voiced_ratio = float(np.sum(vflag) / len(vflag))
    except Exception:
        pass

    return {
        "rms_energy": rms,
        "zcr": zcr,
        "duration": duration,
        "speaking_rate": speaking_rate,
        "f0_mean": f0_mean,
        "f0_std": f0_std,
        "voiced_ratio": voiced_ratio,
    }

segment_emotions = []
for seg in segments_to_use:
    start, end, text = seg["start"], seg["end"], seg["text"]
    w = slice_audio(y_ser, 16000, start, end)

    feats = extract_acoustic_features(w, 16000, text)
    pred = predict_emotion(w, 16000)

    segment_emotions.append({
        "start": start,
        "end": end,
        "text": text,
        "speaker": seg.get("speaker", None),
        "emotion_raw": pred["label"],   # HuBERT 원본 라벨
        "confidence": pred["score"],
        "probs": pred["probs"],         # 감정 확률 분포
        **feats,
    })

print("\n[DEBUG] SER 예시 5개:")
for s in segment_emotions[:5]:
    print(
        f"[{s['start']:.2f}-{s['end']:.2f}] spk={s['speaker']} "
        f"{s['emotion_raw']} ({s['confidence']:.2f}) :: {s['text']}"
    )
print(f"... 총 {len(segment_emotions)}개 세그먼트")



# ============================
# 6. 텍스트 감성 분석 (XLM-R sentiment)
# ============================
txt_model_id = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tok = AutoTokenizer.from_pretrained(txt_model_id)
txt_model = AutoModelForSequenceClassification.from_pretrained(txt_model_id)
txt_model.eval()

id2label_txt = {0: "negative", 1: "neutral", 2: "positive"}

device = "cuda" if torch.cuda.is_available() else "cpu"
txt_model.to(device)

def text_sentiment(s):
    if not s.strip():
        return {"label": "neutral", "score": 0.0}
    inputs = tok(s, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = txt_model(**inputs).logits
    probs = torch.softmax(logits, dim=-1).squeeze().tolist()
    idx = int(np.argmax(probs))
    return {"label": id2label_txt[idx], "score": float(probs[idx])}

for it in segment_emotions:
    ts = text_sentiment(it["text"])
    it["text_sentiment"] = ts["label"]
    it["text_sent_conf"] = ts["score"]



# ============================
# 7. 회의용 감정/태도/상태 태그 + 음성/텍스트 융합
# ============================

# 7-1. HuBERT 라벨 → valence/arousal 스코어 매핑 (기대값으로 사용)
AUDIO_VALENCE_RAW = {
    "happiness": 0.8,
    "sadness": -0.8,
    "anger": -0.7,
    "disgust": -0.7,
    "fear": -0.6,
    "surprise": 0.3,
    "neutral": 0.0,
}
AUDIO_AROUSAL_RAW = {
    "happiness": 0.7,
    "sadness": 0.2,
    "anger": 0.9,
    "disgust": 0.8,
    "fear": 0.9,
    "surprise": 0.8,
    "neutral": 0.3,
}

SENTIMENT_VALENCE = {
    "positive": 1.0,
    "neutral": 0.0,
    "negative": -1.0,
}

# HuBERT 라벨 remap (표기 통일용)
label_map = {
    "happiness": "happy",
    "sadness": "sad",
    "anger": "angry",
    "disgust": "disgust",
    "fear": "fear",
    "surprise": "surprise",
    "neutral": "neutral",
}

def remap(label):
    if label is None:
        return "unknown"
    l = label.lower()
    return label_map.get(l, l)

# valence / arousal / fused 계산
for it in segment_emotions:
    probs = it.get("probs", {}) or {}
    audio_val = 0.0
    audio_arousal = 0.0
    if probs:
        for lb, p in probs.items():
            kk = lb.lower()
            audio_val += AUDIO_VALENCE_RAW.get(kk, 0.0) * p
            audio_arousal += AUDIO_AROUSAL_RAW.get(kk, 0.0) * p
    else:
        kk = it["emotion_raw"].lower()
        audio_val = AUDIO_VALENCE_RAW.get(kk, 0.0) * it.get("confidence", 0.0)
        audio_arousal = AUDIO_AROUSAL_RAW.get(kk, 0.0) * it.get("confidence", 0.0)

    it["audio_valence"] = audio_val
    it["audio_arousal"] = audio_arousal

    txt_label = it.get("text_sentiment", "neutral")
    txt_conf = it.get("text_sent_conf", 0.0)
    txt_val = SENTIMENT_VALENCE.get(txt_label, 0.0) * txt_conf
    it["text_valence"] = txt_val

    # 음성 + 텍스트 융합 (late fusion)
    it["valence_fused"] = (audio_val + txt_val) / 2.0

    # 최종 통합 감정(긍/부/중립) 태그
    vf = it["valence_fused"]
    if vf > 0.25:
        it["fused_sentiment"] = "overall_positive"
    elif vf < -0.25:
        it["fused_sentiment"] = "overall_negative"
    else:
        it["fused_sentiment"] = "overall_neutral"

    # remap된 오디오 감정 레이블도 같이 저장
    it["emotion"] = remap(it["emotion_raw"])


# 7-2. 회의용 세부 태그: 정서/태도/상태
def get_thresholds(arr):
    arr = np.array(arr, dtype=float)
    arr = arr[~np.isnan(arr)]
    if arr.size == 0:
        return 0.0, 0.0, 0.0
    low = float(np.quantile(arr, 0.33))
    med = float(np.median(arr))
    high = float(np.quantile(arr, 0.66))
    return low, med, high

all_rms = [it["rms_energy"] for it in segment_emotions]
all_rate = [it["speaking_rate"] for it in segment_emotions]
all_zcr = [it["zcr"] for it in segment_emotions]
all_f0_std = [it["f0_std"] for it in segment_emotions]

low_rms, med_rms, high_rms = get_thresholds(all_rms)
low_rate, med_rate, high_rate = get_thresholds(all_rate)
low_zcr, med_zcr, high_zcr = get_thresholds(all_zcr)
low_f0s, med_f0s, high_f0s = get_thresholds(all_f0_std)

def classify_meeting_labels(it):
    """
    회의/발표 상황을 위한 3축 태그:
      - affect_label (정서): 중립 / 긍정/낙관 / 부정/우려 / 짜증/불만
      - attitude_label (태도): 확신/단정 / 회의적/의심 / 방어적 / 공감/배려 / 중립
      - state_label (상태): 긴장 / 피곤/권태 / 몰입/열정 / 보통
    """

    v = it["valence_fused"]
    rms = it["rms_energy"]
    rate = it["speaking_rate"]
    zcr = it["zcr"]
    f0_std = it["f0_std"]
    text = it["text"]
    txt_sent = it.get("text_sentiment", "neutral")

    # 간단한 텍스트 기반 시그널
    txt_lower = text.lower()
    is_question = text.strip().endswith("?") or any(k in text for k in ["인가요", "일까요", "을까요", "나요"])
    has_thanks = ("감사" in text) or ("고맙" in text)
    has_blame = ("제 책임" in text) or ("제 잘못" in text) or ("제 탓" in text)
    has_not_my_fault = "아니고" in text or "탓이 아니다" in text
    disfluencies = text.count("어") + text.count("음") + text.count("저기")

    # ---------- 1) 정서(affect): 중립 / 긍정/낙관 / 부정/우려 / 짜증/불만 ----------
    if v > 0.25:
        affect = "긍정/낙관"
    elif v < -0.25:
        # 부정인데 말 속도나 zcr이 높으면 짜증/불만 쪽으로
        if rate > high_rate or zcr > high_zcr:
            affect = "짜증/불만"
        else:
            affect = "부정/우려"
    else:
        affect = "중립"

    # ---------- 2) 태도(attitude): 확신/단정 / 회의적/의심 / 방어적 / 공감/배려 / 중립 ----------
    attitude = "중립"
    if (v > 0.2 and rms > med_rms) or any(kw in text for kw in ["합니다.", "맞습니다", "확실", "분명"]):
        attitude = "확신/단정"
    if is_question and (txt_sent == "negative" or v < 0):
        attitude = "회의적/의심"
    if has_blame or (txt_sent == "negative" and has_not_my_fault):
        attitude = "방어적"
    if txt_sent == "positive" or has_thanks or ("좋은 지적" in text) or ("좋은 의견" in text):
        attitude = "공감/배려"

    # ---------- 3) 상태(state): 긴장 / 피곤/권태 / 몰입/열정 / 보통 ----------
    state = "보통"
    # 긴장: 피치 변동 큼 + 말더듬 + 속도 다소 빠름
    if (f0_std > high_f0s and v <= 0.2) or (disfluencies >= 2 and rate > med_rate):
        state = "긴장"
    # 피곤/권태: 에너지 낮고 속도 느리고 valence 거의 없는 상태
    if rms < low_rms and rate < med_rate and abs(v) <= 0.25:
        state = "피곤/권태"
    # 몰입/열정: 에너지 높고 속도 빠르고 valence 절대값 큼
    if rms > high_rms and rate > high_rate and abs(v) >= 0.25:
        state = "몰입/열정"

    return affect, attitude, state

for it in segment_emotions:
    affect, attitude, state = classify_meeting_labels(it)
    it["affect_label"] = affect
    it["attitude_label"] = attitude
    it["state_label"] = state



# ============================
# 8. 전체 요약 + CSV 저장
# ============================
def summarize(items, key="emotion"):
    seq = [it[key] for it in items if it.get(key) and it[key] != "unknown"]
    if not seq:
        return "No clear label for key=" + key
    counts = Counter(seq)
    top, cnt = counts.most_common(1)[0]
    preview = " → ".join(seq[:5] + (["..."] if len(seq) > 5 else []))
    return f"Dominant({key}): {top} (count={cnt}). Early trend: {preview}"

print("\n[오디오 감정(HuBERT) 요약]", summarize(segment_emotions, key="emotion"))
print("[텍스트 감성(XLM-R) 요약]", summarize(segment_emotions, key="text_sentiment"))
print("[융합 감정(fused_sentiment) 요약]", summarize(segment_emotions, key="fused_sentiment"))
print("[정서(affect) 요약]", summarize(segment_emotions, key="affect_label"))
print("[태도(attitude) 요약]", summarize(segment_emotions, key="attitude_label"))
print("[상태(state) 요약]", summarize(segment_emotions, key="state_label"))

csv_path = "/content/drive/MyDrive/output/ser_segments_meeting.csv"
Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow([
        "start","end","speaker",
        "emotion_raw","emotion","confidence",
        "text_sentiment","text_sent_conf",
        "audio_valence","audio_arousal","text_valence","valence_fused","fused_sentiment",
        "affect_label","attitude_label","state_label",
        "rms_energy","zcr","duration","speaking_rate","f0_mean","f0_std","voiced_ratio",
        "text",
    ])
    for it in segment_emotions:
        w.writerow([
            f"{it['start']:.2f}",
            f"{it['end']:.2f}",
            it.get("speaker", ""),
            it.get("emotion_raw",""),
            it.get("emotion",""),
            f"{it.get('confidence',0.0):.4f}",
            it.get("text_sentiment",""),
            f"{it.get('text_sent_conf',0.0):.4f}",
            f"{it.get('audio_valence',0.0):.4f}",
            f"{it.get('audio_arousal',0.0):.4f}",
            f"{it.get('text_valence',0.0):.4f}",
            f"{it.get('valence_fused',0.0):.4f}",
            it.get("fused_sentiment",""),
            it.get("affect_label",""),
            it.get("attitude_label",""),
            it.get("state_label",""),
            f"{it.get('rms_energy',0.0):.6f}",
            f"{it.get('zcr',0.0):.6f}",
            f"{it.get('duration',0.0):.3f}",
            f"{it.get('speaking_rate',0.0):.3f}",
            f"{it.get('f0_mean',0.0):.3f}",
            f"{it.get('f0_std',0.0):.3f}",
            f"{it.get('voiced_ratio',0.0):.3f}",
            it["text"],
        ])
print("Saved:", csv_path)


Mounted at /content/drive
total 16K
dr-x------ 4 root root 4.0K Nov 28 07:11 .Encrypted
drwx------ 2 root root 4.0K Nov 28 07:11 MyDrive
dr-x------ 2 root root 4.0K Nov 28 07:11 .shortcut-targets-by-id
drwx------ 5 root root 4.0K Nov 28 07:11 .Trash-0
ls: /content/drive/MyDrive/멀티캠퍼스_LLMs_Llama_index: No such file or directory
total 265K
-rw------- 1 root root  176 Nov 21 02:43 '11 21 헤세드 인적사항 사본.gsheet'
-rw------- 1 root root  176 Mar 29  2025 '5 11 업데이트 버스킹 Song List 추천.gsheet'
drwx------ 2 root root 4.0K May  7  2025 'Colab Notebooks'
drwx------ 2 root root 4.0K Nov 13 06:40  data_MP3
drwx------ 2 root root 4.0K Nov 13 06:40  data_MP4
drwx------ 2 root root 4.0K Nov 13 07:30  output
lrw------- 1 root root    0 Nov 19 02:15  멀티캠퍼스_LLMs_Llama_index -> /content/drive/.shortcut-targets-by-id/1L_SL2y6c8AjDF5Plju0IaQnV4pi-pBmJ/멀티캠퍼스_LLMs_Llama_index
-rw------- 1 root root 247K Nov 14 08:04  퍼실AI_baseline.ipynb
-rw------- 1 root root  176 Nov 17 11

/tmp/ipython-input-1425686517.py:247: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


[DEBUG] 원본 오디오 duration = 2522.99 s
[DEBUG] 정제된 오디오 duration = 2518.37 s
RTZR CLIENT_ID를 입력하세요: wtVlucULYnE0c0ATqMlq
RTZR CLIENT_SECRET를 입력하세요 (입력 시 화면에 표시되지 않습니다): ··········
[DEBUG] RTZR STT config:
{
  "model_name": "sommers",
  "domain": "GENERAL",
  "use_diarization": true,
  "diarization": {
    "spk_count": 0
  },
  "use_paragraph_splitter": true,
  "paragraph_splitter": {
    "max": 80
  },
  "use_itn": false,
  "use_disfluency_filter": false,
  "use_profanity_filter": false
}
[INFO] STT 작업 ID: PPadMvcuTzSYlhznTjXQaA
[INFO] 결과 대기 중 (5초 간격 폴링)...
[DEBUG] RTZR status = completed
[DEBUG] 총 utterance 개수 = 276
[DEBUG] 마지막 세그먼트 end = 2517.80 s

[DEBUG] 처음 5개 세그먼트:
  [0.16-8.55] Speaker=0 :: 어 벗어나서 조금 더 이제 탄력적으로 토론을 하는 방식을 조금 실험을 한번 해보고 싶은 것도 있어서 그런 거니까.
  [11.13-26.66] Speaker=0 :: 네, 그러면 두 가지인데 지금 나온 이야기를 좀 종합해 보자면 강점이 그 강점을 가지고 있는 게 소수일 수 있다. 그리고 그 강점이 한편으로는 실행에 급급했던 것이 있고.
  [27.00-39.53] Speaker=0 :: 뭐 그 외에 실행에 수반되는 뭐 명확한 가설이라든가 분석이라든가 혹은 뭐 회고라든가 이런 부분들이 좀 약했다. 그까 그 강점을 강점대로 두되 그

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]


[오디오 감정(HuBERT) 요약] Dominant(emotion): sad (count=197). Early trend: sad → sad → sad → sad → sad → ...
[텍스트 감성(XLM-R) 요약] Dominant(text_sentiment): neutral (count=179). Early trend: neutral → neutral → negative → negative → negative → ...
[융합 감정(fused_sentiment) 요약] Dominant(fused_sentiment): overall_neutral (count=213). Early trend: overall_neutral → overall_neutral → overall_neutral → overall_neutral → overall_negative → ...
[정서(affect) 요약] Dominant(affect_label): 중립 (count=213). Early trend: 중립 → 중립 → 중립 → 중립 → 짜증/불만 → ...
[태도(attitude) 요약] Dominant(attitude_label): 중립 (count=222). Early trend: 중립 → 중립 → 중립 → 중립 → 중립 → ...
[상태(state) 요약] Dominant(state_label): 보통 (count=143). Early trend: 보통 → 피곤/권태 → 보통 → 피곤/권태 → 몰입/열정 → ...
Saved: /content/drive/MyDrive/output/ser_segments_meeting.csv
